In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold
# from sklearn.metrics import accuracy_score

In [3]:
# 讀取測試資料
data = pd.read_csv(r'CleanTest5.csv')
data

,Date,Dow,France,Germany,S&P 500,Hong Kong,NASDAQ,Korea,Japan,Shanghai,Philadelphia,Taiwan,Future,OpenChange,selfDef,kMeans
0,2014/7/2,0.12,0.32,0.09,0.07,1.55,-0.02,0.81,0.29,0.44,0.17,0.46,0.23,-16.18,0,0
1,2014/7/3,0.54,1.21,1.19,0.55,-0.08,0.63,-0.21,-0.14,0.19,0.74,0.44,0.59,-0.30,0,0
2,2014/7/7,-0.26,-1.41,-1.03,-0.39,-0.02,-0.77,-0.23,-0.37,0.03,-0.68,0.11,-0.02,-5.02,0,0
3,2014/7/8,-0.69,-1.40,-1.35,-0.70,0.00,-1.35,0.08,-0.42,0.20,-0.54,0.11,0.28,-20.98,1,1
4,2014/7/9,0.47,0.40,0.36,0.46,-1.55,0.63,-0.31,-0.08,-1.23,0.61,-0.43,-0.13,22.91,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,2021/1/5,0.55,-0.44,-0.55,0.71,0.56,0.95,1.57,-0.37,0.73,1.94,0.66,0.40,131.89,4,4
1303,2021/1/6,1.44,1.19,1.76,0.57,0.29,-0.61,-0.75,-0.38,0.63,-0.32,-0.11,0.80,76.39,3,4
1304,2021/1/7,0.69,0.70,0.55,1.48,-0.24,2.56,2.14,1.60,0.71,3.86,1.54,1.10,151.13,4,4
1305,2021/1/8,0.18,0.65,0.58,0.55,1.07,1.03,3.97,2.36,-0.17,-0.02,1.64,-0.21,-38.37,1,1


In [4]:
# 重新將資料分成三群 (跌、盤整、漲)
def change(x):
    if x < -20:
        y = 2
    elif -20 <= x < 20:
        y = 0
    else :
        y = 1
    return y
data['selfDef2'] = data['OpenChange'].apply(change)
data

,Date,Dow,France,Germany,S&P 500,Hong Kong,NASDAQ,Korea,Japan,Shanghai,Philadelphia,Taiwan,Future,OpenChange,selfDef,kMeans,selfDef2
0,2014/7/2,0.12,0.32,0.09,0.07,1.55,-0.02,0.81,0.29,0.44,0.17,0.46,0.23,-16.18,0,0,0
1,2014/7/3,0.54,1.21,1.19,0.55,-0.08,0.63,-0.21,-0.14,0.19,0.74,0.44,0.59,-0.30,0,0,0
2,2014/7/7,-0.26,-1.41,-1.03,-0.39,-0.02,-0.77,-0.23,-0.37,0.03,-0.68,0.11,-0.02,-5.02,0,0,0
3,2014/7/8,-0.69,-1.40,-1.35,-0.70,0.00,-1.35,0.08,-0.42,0.20,-0.54,0.11,0.28,-20.98,1,1,2
4,2014/7/9,0.47,0.40,0.36,0.46,-1.55,0.63,-0.31,-0.08,-1.23,0.61,-0.43,-0.13,22.91,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,2021/1/5,0.55,-0.44,-0.55,0.71,0.56,0.95,1.57,-0.37,0.73,1.94,0.66,0.40,131.89,4,4,1
1303,2021/1/6,1.44,1.19,1.76,0.57,0.29,-0.61,-0.75,-0.38,0.63,-0.32,-0.11,0.80,76.39,3,4,1
1304,2021/1/7,0.69,0.70,0.55,1.48,-0.24,2.56,2.14,1.60,0.71,3.86,1.54,1.10,151.13,4,4,1
1305,2021/1/8,0.18,0.65,0.58,0.55,1.07,1.03,3.97,2.36,-0.17,-0.02,1.64,-0.21,-38.37,1,1,2


In [5]:
# x = data.iloc[:,1:-4]
x = data.loc[:,['Philadelphia','NASDAQ','S&P 500','Dow','Future','Taiwan','France','Japan','Korea','Germany']]
y = data.loc[:,'selfDef2']

In [6]:
# 訓練集:測試集 -> 7:3
x_train,x_test, y_train, y_test =train_test_split(x, y ,test_size=0.3, random_state=0)

In [7]:
# 利用StratifiedKFold做交叉驗證，相較於KFold，StratifiedKFold會照比例每個data set中抽取資料作驗證
sk_fold = StratifiedKFold(10,shuffle=True, random_state=0)

In [10]:
# GridSearchCV 自動調參
param = ['brute','kd_tree','ball_tree','auto']  # 想要調參的範圍  (實際測過後, 這邊的參數分數都一樣)
parameters ={'algorithm':param} # 想要調參的值
gs_rf = GridSearchCV(estimator=KNeighborsClassifier(), 
                     param_grid=parameters, cv=sk_fold,)
gs_rf.fit(x_train,y_train)

# (最佳結果的參數, 最佳結果的分數)
print('best_params:', gs_rf.best_params_, gs_rf.best_score_)

best_params: {'algorithm': 'kd_tree'} 0.6006330625895844


In [11]:
param = range(1,30)
parameters ={'n_neighbors':param}
gs_rf = GridSearchCV(estimator=KNeighborsClassifier(), 
                     param_grid=parameters, cv=sk_fold )
gs_rf.fit(x_train,y_train)


print('best_params:', gs_rf.best_params_, gs_rf.best_score_)

best_params: {'n_neighbors': 20} 0.6334567606306736


In [14]:
param = range(1,60) 
parameters ={'leaf_size':param}
gs_rf = GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=20), 
                     param_grid=parameters, cv=sk_fold )
gs_rf.fit(x_train,y_train)


print('best_params:', gs_rf.best_params_, gs_rf.best_score_)

best_params: {'leaf_size': 1} 0.6334567606306736


In [15]:
param = ['uniform','distance'] # 實際測過後, 這邊的參數分數都一樣
parameters ={'weights':param}
gs_rf = GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=20,leaf_size=1), 
                     param_grid=parameters, cv=sk_fold )
gs_rf.fit(x_train,y_train)


print('best_params:', gs_rf.best_params_, gs_rf.best_score_)

best_params: {'weights': 'distance'} 0.6334687052078357


In [21]:
param = ['uniform','distance'] 
param2 = range(1,30)
parameters ={'weights':param, 'p':param2}
gs_rf = GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=20,leaf_size=1), 
                     param_grid=parameters, cv=sk_fold )
gs_rf.fit(x_train,y_train)


print('best_params:', gs_rf.best_params_, gs_rf.best_score_)

best_params: {'p': 2, 'weights': 'distance'} 0.6334687052078357


In [9]:
# KNN
model = KNeighborsClassifier(n_neighbors=20,leaf_size=1,p=2,weights='distance')

In [10]:
# 訓練模型
model.fit(x_train,y_train)
y_pred =model.predict(x_test)
print('訓練集準確率:' ,model.score(x_train,y_train))
print('測試集準確率:', model.score(x_test,y_test))

訓練集準確率: 1.0
測試集準確率: 0.6590330788804071


In [11]:
# 混淆矩陣
cm = confusion_matrix(y_test,y_pred)
cm

array([[162,  18,   7],
       [ 62,  64,   0],
       [ 45,   2,  33]], dtype=int64)

In [12]:
# 模型評估報告 (精確度、召回率、F1值、樣本數)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.87      0.71       187
           1       0.76      0.51      0.61       126
           2       0.82      0.41      0.55        80

    accuracy                           0.66       393
   macro avg       0.73      0.60      0.62       393
weighted avg       0.70      0.66      0.65       393



In [13]:
# 讀取測試資料
data2 = pd.read_csv(r'CleanTest.csv')
data2

,Date,Dow,France,Germany,S&P 500,Hong Kong,NASDAQ,Korea,Japan,Shanghai,Philadelphia,Taiwan,Future,OpenChange,selfDef,kMeans
0,2013/1/8,-0.41,0.25,-0.48,-0.32,-0.94,-0.23,0.04,-0.86,-0.41,-1.12,-0.43,-0.47,4.74,0,0
1,2013/1/9,0.46,0.66,0.32,0.27,0.46,0.45,-0.31,0.67,-0.03,1.05,0.22,0.40,42.34,3,3
2,2013/1/10,0.60,-0.01,-0.16,0.76,0.59,0.51,0.75,0.70,0.37,1.33,0.94,1.27,31.93,3,3
3,2013/1/11,0.13,0.32,0.09,0.00,-0.39,0.12,-0.50,1.40,-1.78,0.53,0.10,-0.43,-23.47,1,1
4,2013/1/15,0.20,-0.18,-0.69,0.11,-0.14,-0.22,-1.16,0.72,0.60,-0.50,-0.75,-1.05,1.07,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,2014/6/24,-0.70,0.53,0.17,-0.64,0.33,-0.42,0.98,0.05,0.47,-0.73,0.19,0.05,3.93,0,0
288,2014/6/25,0.29,-1.16,-0.71,0.49,-0.06,0.68,-0.63,-0.71,-0.41,0.38,-0.04,-0.17,31.20,3,3
289,2014/6/26,-0.13,-0.19,-0.64,-0.12,1.45,-0.02,0.67,0.27,0.65,-0.41,0.85,0.84,-4.05,0,0
290,2014/6/27,0.03,0.56,0.10,0.19,0.10,0.43,-0.33,-1.39,-0.11,0.16,-0.15,-0.25,25.61,3,3


In [14]:
# 重新將資料分成三群 (跌、盤整、漲)
data2['selfDef2'] = data2['OpenChange'].apply(change)
data2

,Date,Dow,France,Germany,S&P 500,Hong Kong,NASDAQ,Korea,Japan,Shanghai,Philadelphia,Taiwan,Future,OpenChange,selfDef,kMeans,selfDef2
0,2013/1/8,-0.41,0.25,-0.48,-0.32,-0.94,-0.23,0.04,-0.86,-0.41,-1.12,-0.43,-0.47,4.74,0,0,0
1,2013/1/9,0.46,0.66,0.32,0.27,0.46,0.45,-0.31,0.67,-0.03,1.05,0.22,0.40,42.34,3,3,1
2,2013/1/10,0.60,-0.01,-0.16,0.76,0.59,0.51,0.75,0.70,0.37,1.33,0.94,1.27,31.93,3,3,1
3,2013/1/11,0.13,0.32,0.09,0.00,-0.39,0.12,-0.50,1.40,-1.78,0.53,0.10,-0.43,-23.47,1,1,2
4,2013/1/15,0.20,-0.18,-0.69,0.11,-0.14,-0.22,-1.16,0.72,0.60,-0.50,-0.75,-1.05,1.07,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,2014/6/24,-0.70,0.53,0.17,-0.64,0.33,-0.42,0.98,0.05,0.47,-0.73,0.19,0.05,3.93,0,0,0
288,2014/6/25,0.29,-1.16,-0.71,0.49,-0.06,0.68,-0.63,-0.71,-0.41,0.38,-0.04,-0.17,31.20,3,3,1
289,2014/6/26,-0.13,-0.19,-0.64,-0.12,1.45,-0.02,0.67,0.27,0.65,-0.41,0.85,0.84,-4.05,0,0,0
290,2014/6/27,0.03,0.56,0.10,0.19,0.10,0.43,-0.33,-1.39,-0.11,0.16,-0.15,-0.25,25.61,3,3,1


In [15]:
# x2 = data2.iloc[:,1:-4]
x2 = data2.loc[:,['Philadelphia','NASDAQ','S&P 500','Dow','Future','Taiwan','France','Japan','Korea','Germany']]
y2 = data2.loc[:,'selfDef2']

In [16]:
# 訓練模型
y2_pred =model.predict(x2)
print('準確率:', model.score(x2,y2))

準確率: 0.6095890410958904


In [17]:
# 混淆矩陣
cm2 = confusion_matrix(y2,y2_pred)
cm2

array([[126,   7,   4],
       [ 74,  38,   1],
       [ 28,   0,  14]], dtype=int64)

In [18]:
# 模型評估報告 (精確度、召回率、F1值、樣本數)
print(classification_report(y2, y2_pred))

              precision    recall  f1-score   support

           0       0.55      0.92      0.69       137
           1       0.84      0.34      0.48       113
           2       0.74      0.33      0.46        42

    accuracy                           0.61       292
   macro avg       0.71      0.53      0.54       292
weighted avg       0.69      0.61      0.58       292

